In [7]:
from transformers import AutoModel, PreTrainedModel, TrainingArguments, Trainer, AutoModelForSequenceClassification
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import Dataset
from torch import nn
import pandas as pd
import torch

In [8]:
# Read from CSV
dataset = pd.read_csv("./data/train.csv")

# Drop (potentially) unnecessary columns. These may be useful, but I'm not quite ready to work with missing data.
dataset = dataset.drop(["id", "keyword", "location"], axis=1)
dataset.head(20)

# Init tokenizer for converting text to numbers
model_path = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# In order to add padding on a batch-level rather than a dataset level, add dynamic padding using a data 
# collator. This will add padding to the maximum input in a batch rather than the entire 
# data set which saves computation. 
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Read data from CSV, embed, and split into test and train
raw_dataset = Dataset.from_pandas(dataset)
raw_dataset = raw_dataset.rename_column("target", "labels")
raw_dataset = raw_dataset.map(lambda example: tokenizer(example["text"]), batched=True)
raw_dataset = raw_dataset.with_format("torch")
formatted_datasets = raw_dataset.train_test_split(0.2)

# Show Output
formatted_datasets

100%|██████████| 8/8 [00:00<00:00, 22.10ba/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 6090
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1523
    })
})

In [9]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [14]:
model_save_path = "./models/sanity"
model.save_pretrained(model_save_path)
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_save_path)

In [12]:
inputs = tokenizer(["The sky is ablaze", "The sky is ablaze"], return_tensors="pt")
print(model(**inputs))
print(loaded_model(**inputs))


SequenceClassifierOutput(loss=None, logits=tensor([[ 1.2165, -0.8891],
        [ 1.2165, -0.8891]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[ 1.2165, -0.8891],
        [ 1.2165, -0.8891]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
